In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

from MMClassifyFunc.train import Trainer
from MMClassifyFunc.models import CustomResNet
from MMClassifyFunc.data_preprocess import get_loader_hdf5
from MMClassifyFunc.data_read import get_data_hdf5_wly
from MMClassifyFunc.visualization import visualize_results, visualize_predict

h5_file_path = "/home/mambauser/MMClassify/data/data_processed_noise_mma.hdf5"
in_channels = 1

samples, labels = get_data_hdf5_wly(
    h5_file_path=h5_file_path,
    in_channels=in_channels,
    # wordIndex=[1],
    # fileIndex=list(range(0,10))+list(range(12,30))+list(range(32,40)),
    # personIndex=list(range(5)),
    # txIndex=list(range(0, 8)),
    useLog=True,
    timeLength=200,
)

print("len(samples): {}".format(len(samples)))
print("len(set(labels)): {}".format(len(set(labels))))

trainloader, testloader = get_loader_hdf5(samples=samples, labels=labels)

# classifier
classifier = CustomResNet(
    in_channels=in_channels,
    num_classes=len(set(labels)),
    weights=models.ResNet18_Weights.DEFAULT,
    model="resnet18",
)

# optimizers
lr = 1e-3
betas = (0.5, 0.99)
optimizer = optim.Adam(classifier.parameters(), lr=lr, betas=betas)
criterion = nn.CrossEntropyLoss()

# train model
NUM_INPUTS = 1
epochs = 50

trainer = Trainer(
    num_inputs=NUM_INPUTS,
    classifier=classifier,
    optimizer=optimizer,
    criterion=criterion,
    print_every=1,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    use_cuda=torch.cuda.is_available(),
    use_scheduler=False,
)

trainer.train(trainloader=trainloader, testloader=testloader, epochs=epochs)

visualize_results(trainer=trainer)

len(samples): 5411
len(set(labels)): 46
train_Y
Number 6: 89, Number 39: 87, Number 33: 75, Number 7: 101, Number 26: 115, Number 19: 67, Number 43: 90, Number 28: 63, Number 12: 59, Number 1: 126, Number 17: 97, Number 40: 101, Number 44: 111, Number 41: 95, Number 34: 115, Number 38: 111, Number 24: 108, Number 16: 101, Number 8: 118, Number 3: 100, Number 21: 64, Number 29: 120, Number 4: 102, Number 36: 112, Number 14: 91, Number 18: 96, Number 31: 115, Number 11: 67, Number 15: 49, Number 42: 102, Number 9: 81, Number 27: 76, Number 45: 129, Number 2: 91, Number 10: 73, Number 5: 112, Number 20: 94, Number 32: 104, Number 30: 124, Number 37: 112, Number 23: 14, Number 25: 70, Number 22: 56, Number 35: 118, Number 0: 105, Number 13: 122
test_Y
Number 43: 18, Number 3: 30, Number 25: 24, Number 28: 13, Number 31: 21, Number 10: 17, Number 40: 25, Number 0: 19, Number 34: 29, Number 9: 27, Number 35: 33, Number 38: 38, Number 44: 33, Number 20: 27, Number 21: 20, Number 42: 25, Numbe

Using cache found in /home/mambauser/.cache/torch/hub/pytorch_vision_v0.10.0


Epoch 1 2.29s: train loss 2.447936, train acc 0.298, test loss 1.662402, test acc 0.496
Epoch 2 1.99s: train loss 1.340932, train acc 0.580, test loss 1.121276, test acc 0.638
Epoch 3 1.99s: train loss 1.005132, train acc 0.682, test loss 1.025619, test acc 0.667
Epoch 4 2.06s: train loss 0.704471, train acc 0.771, test loss 0.814274, test acc 0.735


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from tqdm import tqdm

from MMClassifyFunc.train import Trainer
from MMClassifyFunc.models import CustomResNet
from MMClassifyFunc.data_preprocess import get_loader_hdf5, get_loader_all_hdf5
from MMClassifyFunc.data_read import get_data_hdf5_wly
from MMClassifyFunc.visualization import visualize_results, visualize_predict


h5_file_path = "/home/mambauser/MMClassify/data/data_processed_noise_mma.hdf5"
in_channels = 1

samples, labels = get_data_hdf5_wly(
    h5_file_path=h5_file_path,
    in_channels=in_channels,
    wordIndex=[0],
    # fileIndex=list(range(0,10))+list(range(12,30))+list(range(32,40)),
    # personIndex=[0],
    txIndex=[8,9],
    useLog=True,
    timeLength=100,
)

print("len(samples): {}".format(len(samples)))
print("len(set(labels)): {}".format(len(set(labels))))

# Create dataset and dataloader
dataloader = get_loader_all_hdf5(samples, labels)

# classifier
trainer.classifier.eval()

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# classifier.to(device)
trainer.classifier.to(device)

# Prepare for evaluation
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels in tqdm(dataloader, desc="Processing batches"):
        images, labels = images.to(device), labels.to(device)
        outputs = trainer.classifier(images)
        _, predicted = torch.max(outputs.data, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

visualize_predict(all_labels, all_preds)